In [1]:
import pandas as pd

In [2]:
seine_et_marneDf = pd.read_csv("data/seine_et_marne.csv")
seine_et_marneDf

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,new_commune,Surface Totale
0,1,2018-07-02,Vente,100000.0,92.0,RUE,1610,SAINT DENIS,77400.0,LAGNY-SUR-MARNE,...,417,1,2.0,Appartement,22.0,1,NaN,NaN,77243,22.0
1,1,2018-07-02,Vente,262000.0,NaN,NaN,B025,SAINT GERMAIN,77860.0,SAINT-GERMAIN-SUR-MORIN,...,133,0,NaN,Terrain,NaN,0,S,8.0,77413,8.0
2,1,2018-07-02,Vente,262000.0,5.0,RUE,0520,LOUIS DOUVIZIE,77860.0,SAINT-GERMAIN-SUR-MORIN,...,132,0,1.0,Maison,84.0,3,S,110.0,77413,194.0
3,1,2018-07-02,Vente,262000.0,NaN,NaN,B025,SAINT GERMAIN,77860.0,SAINT-GERMAIN-SUR-MORIN,...,131,0,NaN,Terrain,NaN,0,S,297.0,77413,297.0
4,1,2018-07-02,Vente,262000.0,NaN,NaN,B025,SAINT GERMAIN,77860.0,SAINT-GERMAIN-SUR-MORIN,...,130,0,NaN,Terrain,NaN,0,S,8.0,77413,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381364,1,2023-06-30,Vente,12500.0,28.0,RUE,1610,SAINT DENIS,77400.0,LAGNY-SUR-MARNE,...,145,1,3.0,Dépendance,0.0,0,NaN,NaN,77243,0.0
381365,1,2023-06-30,Vente,790889.0,245.0,AV,0128,DE LA GRANDE HAIE,77100.0,MAREUIL-LES-MEAUX,...,520,0,4.0,Local industriel. commercial ou assimilé,1663.0,0,S,2630.0,77276,4293.0
381366,1,2023-06-30,Vente,790889.0,245.0,AV,0128,DE LA GRANDE HAIE,77100.0,MAREUIL-LES-MEAUX,...,520,0,4.0,Local industriel. commercial ou assimilé,0.0,0,S,2630.0,77276,2630.0
381367,1,2023-06-30,Vente,790889.0,NaN,AV,0128,DE LA GRANDE HAIE,77100.0,MAREUIL-LES-MEAUX,...,523,0,NaN,Terrain,NaN,0,S,1363.0,77276,1363.0


In [44]:
from sklearn.model_selection import train_test_split

numeric_features = ["Surface Totale", "Nombre pieces principales"]
categorical_features = ["new_commune"]
date_features = ["Date mutation"]


def preparation_data(df, numeric_features, categorical_features, date_features, test_size=0.2):
    X = df[numeric_features + categorical_features + date_features]
    y = df["Valeur fonciere"]

    return train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)


X_train, X_test, y_train, y_test = preparation_data(
    seine_et_marneDf, numeric_features, categorical_features, date_features)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [77]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1))])


def get_timestamp(X):
    return pd.to_datetime(X.values[0], format="mixed", dayfirst=False).astype(int) // 10 ** 11


date_transformer = Pipeline(steps=[
    ('timestamp', FunctionTransformer(get_timestamp))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('date', date_transformer, date_features)], remainder='drop')

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', HistGradientBoostingRegressor(categorical_features=categorical_features, random_state=42))], verbose=True)
# ('regressor', HistGradientBoostingRegressor(random_state=42))], verbose=True)

model.fit(X_train, y_train)

# Results
y_pred = model.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("Score: %.2f" % model.score(X_test, y_test))
model

ValueError: The output of the 'date' transformer should be 2D (scipy matrix, array, or pandas DataFrame).